In [3]:
import pandas as pd
from OrcFxAPI import*
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#TENSION AND ASSOCIATED MBR CURVE DATA
#format 1st line: "MBR", "Tension"
#format 2nd line: mbr_value_1, tension_value_1
#format nth line: mbr_value_n, tension_value_n

i_file = open('input_data.csv')
data = i_file.readlines()
i_file.close()

mbr=[]
tens=[]
for line in data[1:]:
    mbr.append(float(line.split(',')[0]))
    tens.append(float(line.split(',')[1]))

print(mbr,tens)

[11.8, 12.83, 14.05, 15.54, 17.37, 19.7, 22.75, 26.75, 32.78, 42.37, 59.96, 100.64, 298.25] [0.0, 100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0, 1100.0, 1200.0]


In [5]:
#get umbilical information
umb_OD = 0.215 #m
umb_EA = 773   #MN
E_Mod = 200e3 #MPA

In [6]:
#get tube data
SMYS = 620 #MPa
tube_ID = 0.0381 #m
wall_thickness = 0.004 #m

p_i = 68.9 #MPA
p_o = 0.0  #MPA

tube_OD = tube_ID + 2.0*wall_thickness

In [7]:
#project specific data
wd = 1790 #m
condition = "Normal" #possible values: "Normal" or "Abnormal"

if condition == "Normal":
    tau = 0.8
elif condition == "Abnormal":
    tau = 1.0

In [8]:
a = 1.0
[a][0]

1.0

In [15]:
#STRESS FUNCTIONS

# class stresses:

#     def __init__(self):
#         mbr = []
#         tens = []

#     def tube(self,name, p_i, p_o, tube_ID,tube_OD,wall_thickness):
#         self.name = name
#         self.p_i = p_i
#         self.p_o = p_o
#         self.tube_ID = tube_ID
#         self.tube_OD = tube_OD
#         self.wall_thickness = wall_thickness
    
#     def umb(self, umb_EA, umb_OD, E_Mod):
#         self.umb_EA = umb_EA
#         self.umb_OD = umb_OD
#         self.E_Mod = E_Mod

#end cap stress
def stress_aec() -> float:
    """ calculates the end cap stress, not dependent on the tension vs MBR values """
    return ((p_i-p_o)*(tube_OD-2.0*wall_thickness)**2) / (tube_OD**2 - (tube_OD - 2*wall_thickness)**2)


#stress due to tensile loads
def stress_at(tens) -> list:
    """ calculates the stress due to tensile loads """
    return [E_Mod * 0.001 * i/umb_EA for i in [tens]]


#INSIDE WALL
#stress inside the wall
def stress_bi(mbr) -> list:
    """ Calculates the stress due to bending inside the wall of the tube """
    return [E_Mod * ((0.5 * tube_ID) / i) for i in [mbr]]


#hoop stress inside the wall
def stress_hi() -> float:
    """ calculates teh hoop stress inside the wall """
    return ((p_i - p_o)*(tube_OD**2.0 + tube_ID**2.0)) / (tube_OD**2.0 - tube_ID**2.0)-p_o

#radial strees inside the wall
def stress_ri() -> float:
    """ Calculates the radial stress inside the wall """
    return -p_i

#total axial stress inside
def total_axial_stress_inside(mbr, tens) -> list:
    """ Caculates the total axial stress inside the wall """
    return [stress_aec() + stress_at([tens][i])[0] + stress_bi([mbr][i])[0] for i in range(len([tens]))] 

#equivalent stress inside
def stress_eqi(mbr, tens) -> list:
    """ Calculates the inside equivalent stress """
    return [ np.sqrt((stress_hi()-total_axial_stress_inside(mbr[i],tens[i])[0])**2.0 + (total_axial_stress_inside(mbr[i],tens[i])[0]-stress_ri())**2.0 + (stress_ri() - stress_hi())**2.0) / np.sqrt(2.0) for i in range(len(tens))]

stress_eqi(mbr,tens)



#OUTSIDE WALL
#stress inside the wall
def stress_bo(mbr) -> list:
    """ Calculates the stress due to bending inside the wall of the tube """
    return [E_Mod * ((0.5 * tube_OD) / i) for i in [mbr]]


#hoop stress inside the wall
def stress_ho() -> float:
    """ calculates teh hoop stress inside the wall """
    return (p_i - p_o)*((2.0*tube_ID**2.0) / (tube_OD**2.0 - tube_ID**2.0)) -p_o


#radial strees inside the wall
def stress_ro() -> float:
    """ Calculates the radial stress inside the wall """
    return -p_o

#total axial stress inside
def total_axial_stress_outside(mbr, tens) -> list:
    """ Caculates the total axial stress inside the wall """
    return [stress_aec() + stress_at([tens][i])[0] + stress_bo([mbr][i])[0] for i in range(len([tens]))] 

#equivalent stress inside
def stress_eqo(mbr, tens) -> list:
    """ Calculates the inside equivalent stress """
    return [ np.sqrt((stress_ho()-total_axial_stress_outside(mbr[i],tens[i])[0])**2.0 + (total_axial_stress_outside(mbr[i],tens[i])[0]-stress_ro())**2.0 + (stress_ro() - stress_ho())**2.0) / np.sqrt(2.0) for i in range(len(tens))]

    max(stress_eqo(mbr,tens),stress_eqi(mbr,tens))




[495.9981631907831,
 495.9669924904027,
 496.0238539097666,
 495.9409164999949,
 495.96901701811953,
 495.9238071295574,
 495.8878392220522,
 496.4287744506817,
 496.21553694695797,
 495.9329088127436,
 495.60337847512443,
 495.72602023357416,
 496.2402860361266]